In [39]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
DATASET_DIR = './data/'
GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1', 'domain1_score', 'essay_id'])
prompt_index = X['essay_set']
# X = X.drop(columns=['essay_set'])
prompts_text = open(os.path.join(DATASET_DIR, 'prompt.txt')).read().splitlines()
X.head()

,essay_set,essay
0,1,"Dear local newspaper, I think effects computer..."
1,1,"Dear @CAPS1 @CAPS2, I believe that using compu..."
2,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl..."
3,1,"Dear Local Newspaper, @CAPS1 I have found that..."
4,1,"Dear @LOCATION1, I know having computers has a..."


In [32]:
from collections import Counter
import re
from nltk.corpus import stopwords
import numpy as np

def tokenize(essays):
  """Tokenize essay into words, remove stopwords and lowercasing"""
  res = []
  for essay in essays:
    essay = re.sub(r'[^\w\s]', '', essay)  
    essay = essay.lower()  
    essay = essay.split()
    essay = [word for word in essay if word not in stopwords.words('english')]
    res.append(essay)
  return res
    
def compute_words_rate(corpus):
  """Compute generic rates G_i from the entire corpus."""
  word_counts = Counter()
  total_words = 0
  for essay in corpus:
    word_counts.update(essay)
    total_words += len(essay)
  return {word: count / total_words for word, count in word_counts.items()}

def compute_psi(essay, generic_rate, prompt_specific_rate):
    """Compute the Prompt-Specific Index for a single essay."""
    N = len(essay)  
    if N == 0:  # Avoid division by zero
        return 0

    psi_sum = 0
    for word in essay:
        G_i = generic_rate.get(word, 0)
        S_i = prompt_specific_rate.get(word, 0)
        psi_sum += np.sqrt(S_i * (1 - G_i))

    return psi_sum / N


In [33]:
X_p = np.array(pd.read_csv(os.path.join(SAVE_DIR, 'X_p.csv')))
tmp = []
for i in range(X_p.shape[0]):
  tmp.append(np.array([x for x in X_p[i] if not pd.isnull(x)]))
X_p = tmp

In [34]:
# np.random.seed(0)
# from sklearn.model_selection import train_test_split
# # train valid split
# X_train, X_valid, y_train, y_valid = train_test_split(X_p, X['essay_set'], test_size=0.2, random_state=0)
# y_train = y_train.tolist()
# y_valid = y_valid.tolist()


# # shuffle y valid
# true = y_valid.copy()
# np.random.shuffle(y_valid)
# true = [1 if x == y else 0 for x, y in zip(true, y_valid)]

# # find number of 0 and 1
# print('Number of 0:', true.count(0))
# print('Number of 1:', true.count(1))
# print(len(true))

In [40]:
# precompute

def modelB(X_train, y_train, novel_essays, novel_essays_prompt):
  generic_rate= compute_words_rate(X_train)
  prompt_rates= []
  for i in range(1, 9):
    indices = [j for j, index in enumerate(y_train) if index == i]
    prompt_corpus =  [X_train[j] for j in indices]
    prompt_rates.append(compute_words_rate(prompt_corpus))
  pred = []
  i = 0
  for essay in novel_essays:
    pred.append(compute_psi(essay, generic_rate, prompt_rates[novel_essays_prompt[i] - 1]))
    i += 1
  return pred

In [53]:
# use cross validation to find optimal threshold
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
kf = KFold(n_splits=20, random_state=0, shuffle=True)
thresholds = []
for train_index, test_index in kf.split(X_p):
  X_train_cv, X_test_cv = [X_p[i] for i in train_index], [X_p[i] for i in test_index]
  y_train_cv, y_test_cv = [prompt_index[i] for i in train_index], [prompt_index[i] for i in test_index]
  true = y_test_cv.copy()
  np.random.shuffle(y_test_cv)
  true = [1 if x == y else 0 for x, y in zip(y_test_cv, true)]
  pred_proba = modelB(X_train_cv, y_train_cv, X_test_cv, y_test_cv)
  fpr, tpr, threshold = roc_curve(true, pred_proba)
  optimal_idx = np.argmax(tpr - fpr)
  thresholds.append(threshold[optimal_idx])
print(np.mean(thresholds))

0.03722947185919021


In [37]:
# stores genereic rate and prompt rates in a csv file
# generic_rate= compute_words_rate(X_p)
# prompt_rates= []
# for i in range(1, 9):
#   indices = [j for j, index in enumerate(X["essay_set"]) if index == i]
#   prompt_corpus =  [X_p[j] for j in indices]
#   prompt_rates.append(compute_words_rate(prompt_corpus))

# import csv
# with open(os.path.join(SAVE_DIR, 'generic_rates.csv'), 'w') as f:
#   writer = csv.writer(f)
#   for key, value in generic_rate.items():
#     writer.writerow([key, value])
# for i in range(1, 9):
#   with open(os.path.join(SAVE_DIR, 'prompt_rates_' + str(i) + '.csv'), 'w') as f:
#     writer = csv.writer(f)
#     for key, value in prompt_rates[i - 1].items():
#       writer.writerow([key, value])

In [38]:
# find threshold for psi 
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from matplotlib import pyplot as plt

pred_proba = modelB(X_valid, y_valid)
fpr, tpr, threshold = roc_curve(true, pred_proba)
roc_auc = auc(fpr, tpr)

roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
roc_display.plot()
plt.show()

# find threshold
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = threshold[optimal_idx]
print(optimal_threshold)

TypeError: modelB() missing 2 required positional arguments: 'novel_essays' and 'novel_essays_prompt'

In [ ]:
# compute accuracy, confusion matrix
pred_proba = modelB(X_valid, y_valid)
pred = np.array([1 if x > optimal_threshold else 0 for x in pred_proba])
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true, pred)
print(accuracy)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true, pred)
print(cm)

0.9691833590138675
[[2143   71]
 [   9  373]]


determine the threshold using the percentile value

In [ ]:
pred_proba = modelB(X_valid, y_valid)
threshold = np.percentile(pred_proba, 90)
print(threshold)
# confusion matrix
pred = np.array([1 if x > threshold else 0 for x in pred_proba])
accuracy = np.mean(pred == true)
print(accuracy)

cm = confusion_matrix(true, pred)
print(cm)

0.05306379340615158
0.9530046224961479
[[2214    0]
 [ 122  260]]
